# Drunken Tower of Hanoi - Problem 497
<p>Bob is very familiar with the famous mathematical puzzle/game, "Tower of Hanoi," which consists of three upright rods and disks of different sizes that can slide onto any of the rods. The game begins with a stack of $n$ disks placed on the leftmost rod in descending order by size. The objective of the game is to move all of the disks from the leftmost rod to the rightmost rod, given the following restrictions:</p>

<ol><li>Only one disk can be moved at a time.</li>
<li>A valid move consists of taking the top disk from one stack and placing it onto another stack (or an empty rod).</li>
<li>No disk can be placed on top of a smaller disk.</li>
</ol><p>Moving on to a variant of this game, consider a long room $k$ units (square tiles) wide, labeled from $1$ to $k$ in ascending order. Three rods are placed at squares $a$, $b$, and $c$, and a stack of $n$ disks is placed on the rod at square $a$.</p>

<p>Bob begins the game standing at square $b$. His objective is to play the Tower of Hanoi game by moving all of the disks to the rod at square $c$. However, Bob can only pick up or set down a disk if he is on the same square as the rod/stack in question.</p>

<p>Unfortunately, Bob is also drunk. On a given move, Bob will either stumble one square to the left or one square to the right with equal probability, unless Bob is at either end of the room, in which case he can only move in one direction. Despite Bob's inebriated state, he is still capable of following the rules of the game itself, as well as choosing when to pick up or put down a disk.</p>

<p>The following animation depicts a side-view of a sample game for $n = 3$, $k = 7$, $a = 2$, $b = 4$, and $c = 6$:</p>

<p align="center"><img src="resources/images/0497_hanoi.gif?1678992057" alt="0497_hanoi.gif"></p>

<p>Let $E(n, k, a, b, c)$ be the expected number of squares that Bob travels during a single optimally-played game. A game is played optimally if the number of disk-pickups is minimized.</p>

<p>Interestingly enough, the result is always an integer. For example, $E(2,5,1,3,5) = 60$ and $E(3,20,4,9,17) = 2358$.</p>

<p>Find the last nine digits of $\sum_{1\le n \le 10000} E(n,10^n,3^n,6^n,9^n)$.</p>

## Solution.

In [157]:
from functools import lru_cache
import numpy as np
from tqdm import tqdm

In [158]:
def modular_exponentiation(a, b, mod):
    result = 1
    a = a % mod
    while b > 0:
        if (b % 2) == 1: 
            result = (result * a) % mod
        a = (a * a) % mod 
        b //= 2 
    return result


In [159]:
@cache
def expected_steps(k, x, y, mod=10**9):
    ''' Finds expected number of steps for Bob to go from x to y'''
    
    if y < x:
        x = (k - x + 1) % mod
        y = (k - y + 1) % mod

    diff = (y - x)%mod
    
    a = 2*diff % mod
    b = diff*(diff+2) % mod
    
    return (a*y - b) % mod

In [160]:
@lru_cache(None)
def number_of_moves(n, x, y, start, mod=10**9):
    ''' Number of moves between rods in Tower of Hanoi when moving a tower of height n from x to y and starting from start'''
    if n == 1:
        ans = {'12': 0, '13': 0, '21': 0, '23': 0, '31': 0, '32': 0}
        first_move = start + x
        second_move = x + y
        if first_move != second_move:
            ans[first_move] += 1
            ans[second_move] += 1
        else:
            ans[first_move] += 1

        return ans

    z = str(6 - int(x) - int(y))
    ans = {'12': 0, '13': 0, '21': 0, '23': 0, '31': 0, '32': 0}
    moves1 = number_of_moves(n-1, x, z, start)
    moves2 = number_of_moves(n-1, z, y, y)

    ans[z+x] += 1
    ans[x+y] += 1
    for m in moves1:
        ans[m] += moves1[m]
        ans[m] += moves2[m]
        ans[m] = ans[m] % mod
    
    return ans


In [161]:
def dic_to_list(dic):
    ans = []
    for d in dic:
        ans.append(dic[d])
    return ans

In [169]:
@lru_cache(None)
def E(n, k, a, b, c, mod=10**9):
    ans = 0
    a_to_b = expected_steps(k, a, b)
    a_to_c = expected_steps(k, a, c)
    b_to_a = expected_steps(k, b, a)
    b_to_c = expected_steps(k, b, c)
    c_to_a = expected_steps(k, c, a)
    c_to_b = expected_steps(k, c, b)

    ab, ac, ba, bc, ca, cb = dic_to_list(number_of_moves(n, '1', '3', '2'))

    ans += a_to_b * ab % mod
    ans += a_to_c * ac % mod
    ans += b_to_a * ba % mod
    ans += b_to_c * bc % mod
    ans += c_to_a * ca % mod
    ans += c_to_b * cb % mod
    
    return ans % mod

In [163]:
E(2, 10**2, 3**2, 6**2, 9**2)

24777

In [192]:
ans = 0
mod = 10**9

for n in tqdm(range(1, 10001)):
    k = 10**n
    a = 3**n
    b = 6**n
    c = 9**n
    ans += E(n, k, a, b, c)
    ans = ans%mod

print(ans)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5525.64it/s]

684901360
